## Scratchbook ##

meant for testing and non production activities

In [2]:
import pandas as pd
import glob
import time
import duckdb
import os
from datetime import datetime, timedelta

In [3]:
dataset = "dataset/JAAR_DB.duckdb"
con = duckdb.connect(database= dataset)

In [4]:
# Get all table names
tables = con.execute("SHOW TABLES").fetchall()

# Preview first few rows of each table
for table in tables:
    table_name = table[0]
    print(f"\n=== {table_name} ===")
    
    # Show first 5 rows
    preview = con.execute(f"SELECT * FROM {table_name} LIMIT 5").fetchall()
    columns = [desc[0] for desc in con.description]
    
    print(f"Columns: {columns}")
    print("Sample data:")
    for row in preview:
        print(row)


=== JAAR_raw ===
Columns: ['TIMESTAMP', 'ChamberID', 'ChamberStatus', 'ChamberTC', 'PPFD', 'note', 'CH4', 'CO2', 'H2O_LI7810', 'N2O', 'H2O_LI7820', 'PS01']
Sample data:
('2025-03-04 00:00:00', 5.0, 0.0, 21.59, 0.0, 'raw', 2034.394, 535.46301, 17906.551, 361.42618, 17856.406, 1007.3)
('2025-03-04 00:00:01', 5.0, 0.0, 21.59, 0.0, 'raw', 2032.8506, 528.85876, 18091.588, 361.50317, 17936.115, 1007.3)
('2025-03-04 00:00:02', 5.0, 0.0, 21.58, 0.0, 'raw', 2027.2987, 518.67578, 18395.146, 359.33124, 18169.088, 1007.3)
('2025-03-04 00:00:03', 5.0, 0.0, 21.59, 0.0, 'raw', 2025.5663, 517.84766, 18508.52, 358.93921, 18267.242, 1007.3)
('2025-03-04 00:00:04', 5.0, 0.0, 21.59, 0.0, 'raw', 2025.6571, 518.44781, 18611.219, 358.69244, 18352.871, 1007.3)


In [5]:
# Query into a pandas DataFrame for better display
df = con.execute("""
    SELECT TIMESTAMP, ChamberID, ChamberStatus, ChamberTC, CH4 
    FROM JAAR_raw 
    LIMIT 50
""").df()

print(df)

              TIMESTAMP  ChamberID  ChamberStatus  ChamberTC        CH4
0   2025-03-04 00:00:00        5.0            0.0      21.59  2034.3940
1   2025-03-04 00:00:01        5.0            0.0      21.59  2032.8506
2   2025-03-04 00:00:02        5.0            0.0      21.58  2027.2987
3   2025-03-04 00:00:03        5.0            0.0      21.59  2025.5663
4   2025-03-04 00:00:04        5.0            0.0      21.59  2025.6571
5   2025-03-04 00:00:05        5.0            0.0      21.59  2025.9663
6   2025-03-04 00:00:06        5.0            0.0      21.58  2026.1731
7   2025-03-04 00:00:07        5.0            0.0      21.59  2026.0664
8   2025-03-04 00:00:08        5.0            0.0      21.58  2026.1788
9   2025-03-04 00:00:09        5.0            0.0      21.59  2026.1869
10  2025-03-04 00:00:10        5.0            0.0      21.60  2026.1912
11  2025-03-04 00:00:11        5.0            0.0      21.58  2025.4135
12  2025-03-04 00:00:12        5.0            0.0      21.59  20

In [6]:
# Observing the chambers available with non 0.0 status
chambers = con.execute("""
    SELECT DISTINCT ChamberID 
    FROM JAAR_raw 
    WHERE ChamberStatus != 0.0
    ORDER BY ChamberID
""").fetchall()

print("Chambers with non-zero status:")
for chamber in chambers:
    print(f"Chamber ID: {chamber[0]}")

Chambers with non-zero status:
Chamber ID: 1.0
Chamber ID: 2.0
Chamber ID: 3.0
Chamber ID: 4.0
Chamber ID: 5.0
Chamber ID: 6.0
Chamber ID: 7.0
Chamber ID: 8.0
Chamber ID: 9.0
